## 1. Импорт библиотек и подготовка данных

In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas as pd
import numpy as np

# Загрузка датасета
data = pd.read_csv("winequality-white.csv", sep=";")

# Создаем бинарную целевую переменную: 1 — хорошее (>=7), 0 — обычное (<7)
data["good"] = (data["quality"] >= 7).astype(int)

X = data.drop(columns=["quality", "good"])
y = data["good"]

# Делим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Стандартизация признаков
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразуем в тензоры
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

print("Форма обучающих данных:", X_train_tensor.shape)
print("Форма тестовых данных:", X_test_tensor.shape)


## 2. Определение архитектуры нейронной сети

In [ ]:

class WineMLP(nn.Module):
    def __init__(self, input_dim):
        super(WineMLP, self).__init__()
        self.hidden1 = nn.Linear(input_dim, 12)
        self.hidden2 = nn.Linear(12, 12)
        self.output = nn.Linear(12, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))
        x = self.sigmoid(self.output(x))
        return x

input_dim = X_train.shape[1]
model = WineMLP(input_dim)
model


## 3. Инициализация модели, функции потерь и оптимизатора

In [ ]:

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## 4. Обучение модели

In [ ]:

EPOCHS = 50
for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f"Эпоха [{epoch+1}/{EPOCHS}], Потери: {loss.item():.4f}")


## 5. Оценка модели

In [ ]:

model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor)
    y_pred_cls = (y_pred > 0.5).float()

acc = accuracy_score(y_test_tensor, y_pred_cls)
f1 = f1_score(y_test_tensor, y_pred_cls)

print("\n--- Результаты модели ---")
print(f"Accuracy: {acc:.4f}")
print(f"F1-score: {f1:.4f}")
print(classification_report(y_test_tensor, y_pred_cls, digits=4, zero_division=0))


## 6. Эксперимент: увеличение числа эпох в 2 раза

In [ ]:

model2 = WineMLP(input_dim)
optimizer2 = optim.Adam(model2.parameters(), lr=0.001)
EPOCHS2 = EPOCHS * 2

for epoch in range(EPOCHS2):
    model2.train()
    optimizer2.zero_grad()
    outputs = model2(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer2.step()

model2.eval()
with torch.no_grad():
    y_pred2 = model2(X_test_tensor)
    y_pred2_cls = (y_pred2 > 0.5).float()

acc2 = accuracy_score(y_test_tensor, y_pred2_cls)
f12 = f1_score(y_test_tensor, y_pred2_cls)

print("\n--- Эксперимент (эпохи x2) ---")
print(f"Accuracy: {acc2:.4f}")
print(f"F1-score: {f12:.4f}")
print(classification_report(y_test_tensor, y_pred2_cls, digits=4, zero_division=0))
